In [1]:
from __future__ import unicode_literals
import textacy
import spacy

nlp = spacy.load("en")
from spacy import displacy
from spacy.compat import *
from textacy import *
from __future__ import absolute_import, division, print_function, unicode_literals

import collections
import itertools
import operator
import re

import numpy as np
from cytoolz import itertoolz
from spacy.parts_of_speech import CONJ, DET, NOUN, VERB
from spacy.tokens.span import Span as SpacySpan

from textacy import compat
from textacy import constants
from textacy import spacy_utils
from textacy import text_utils

#import PyPDF2
import string
import numpy as np
import nltk
from nltk.corpus import stopwords
import re
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer

import numpy
import pandas as pd
import itertools
import xlrd
from xlrd import open_workbook
from itertools import chain,combinations


import logging

from cytoolz import itertoolz
import networkx as nx
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from spacy.tokens.span import Span as SpacySpan
from spacy.tokens.token import Token as SpacyToken

from textacy.compat import unicode_
from textacy import extract
import matplotlib.pyplot as plt

LOGGER = logging.getLogger(__name__)
from __future__ import absolute_import, division, print_function, unicode_literals

from itertools import takewhile
from spacy.parts_of_speech import NOUN, PROPN, VERB
from spacy.tokens.token import Token as SpacyToken
from spacy.tokens.span import Span as SpacySpan

from textacy.text_utils import is_acronym
from textacy.constants import AUX_DEPS, SUBJ_DEPS, OBJ_DEPS

In [16]:
def verbieproc(document):
    
    
    AUX_DEPS1 = {'aux', 'auxpass', 'neg', 'advmod','prt'}
    OBJ_DEPS1 = {'attr', 'dative', 'dobj', 'oprd', 'npadvmod'}
   
    def words(doc,
              filter_stops=True, filter_punct=True, filter_nums=False,
              include_pos=None, exclude_pos=None, min_freq=1):
        """
        Extract an ordered sequence of words from a document processed by spaCy,
        optionally filtering words by part-of-speech tag and frequency.
        Args:
            doc (``textacy.Doc``, ``spacy.Doc``, or ``spacy.Span``)
            filter_stops (bool): if True, remove stop words from word list
            filter_punct (bool): if True, remove punctuation from word list
            filter_nums (bool): if True, remove number-like words (e.g. 10, 'ten')
                from word list
            include_pos (str or Set[str]): remove words whose part-of-speech tag
                IS NOT included in this param
            exclude_pos (str or Set[str]): remove words whose part-of-speech tag
                IS in the specified tags
            min_freq (int): remove words that occur in `doc` fewer than
                `min_freq` times
        Yields:
            ``spacy.Token``: the next token from ``doc`` passing specified filters
            in order of appearance in the document
        Raises:
            TypeError: if `include_pos` or `exclude_pos` is not a str, a set of str,
                or a falsy value
        Note:
            Filtering by part-of-speech tag uses the universal POS tag set,
            http://universaldependencies.org/u/pos/.
        """
        words_ = (w for w in doc if not w.is_space)
        if filter_stops is True:
            words_ = (w for w in words_ if not w.is_stop)
        if filter_punct is True:
            words_ = (w for w in words_ if not w.is_punct)
        if filter_nums is True:
            words_ = (w for w in words_ if not w.like_num)
        if include_pos:
            if isinstance(include_pos, compat.unicode_):
                include_pos = include_pos.upper()
                words_ = (w for w in words_ if w.pos_ == include_pos)
            elif isinstance(include_pos, (set, frozenset, list, tuple)):
                include_pos = {pos.upper() for pos in include_pos}
                words_ = (w for w in words_ if w.pos_ in include_pos)
            else:
                msg = 'invalid `include_pos` type: "{}"'.format(type(include_pos))
                raise TypeError(msg)
        if exclude_pos:
            if isinstance(exclude_pos, compat.unicode_):
                exclude_pos = exclude_pos.upper()
                words_ = (w for w in words_ if w.pos_ != exclude_pos)
            elif isinstance(exclude_pos, (set, frozenset, list, tuple)):
                exclude_pos = {pos.upper() for pos in exclude_pos}
                words_ = (w for w in words_ if w.pos_ not in exclude_pos)
            else:
                msg = 'invalid `exclude_pos` type: "{}"'.format(type(exclude_pos))
                raise TypeError(msg)
        if min_freq > 1:
            words_ = list(words_)
            freqs = itertoolz.frequencies(w.lower_ for w in words_)
            words_ = (w for w in words_
                      if freqs[w.lower_] >= min_freq)

        for word in words_:
            yield word

    def noun_chunks(doc, drop_determiners=True, min_freq=1):
        """
        Extract an ordered sequence of noun chunks from a spacy-parsed doc, optionally
        filtering by frequency and dropping leading determiners.
        Args:
            doc (``textacy.Doc`` or ``spacy.Doc``)
            drop_determiners (bool): remove leading determiners (e.g. "the")
                from phrases (e.g. "the quick brown fox" => "quick brown fox")
            min_freq (int): remove chunks that occur in `doc` fewer than
                `min_freq` times
        Yields:
            ``spacy.Span``: the next noun chunk from ``doc`` in order of appearance
            in the document
        """
        if hasattr(doc, 'spacy_doc'):
            ncs = doc.spacy_doc.noun_chunks
        else:
            ncs = doc.noun_chunks
        if drop_determiners is True:
            ncs = (nc if nc[0].pos != DET else nc[1:]
                   for nc in ncs)
        if min_freq > 1:
            ncs = list(ncs)
            freqs = itertoolz.frequencies(nc.lower_ for nc in ncs)
            ncs = (nc for nc in ncs
                   if freqs[nc.lower_] >= min_freq)

        for nc in ncs:
            yield nc


    def pos_regex_matches(doc, pattern):
        """
        Extract sequences of consecutive tokens from a spacy-parsed doc whose
        part-of-speech tags match the specified regex pattern.
        Args:
            doc (``textacy.Doc`` or ``spacy.Doc`` or ``spacy.Span``)
            pattern (str): Pattern of consecutive POS tags whose corresponding words
                are to be extracted, inspired by the regex patterns used in NLTK's
                `nltk.chunk.regexp`. Tags are uppercase, from the universal tag set;
                delimited by < and >, which are basically converted to parentheses
                with spaces as needed to correctly extract matching word sequences;
                white space in the input doesn't matter.
                Examples (see ``constants.POS_REGEX_PATTERNS``):
                * noun phrase: r'<DET>? (<NOUN>+ <ADP|CONJ>)* <NOUN>+'
                * compound nouns: r'<NOUN>+'
                * verb phrase: r'<VERB>?<ADV>*<VERB>+'
                * prepositional phrase: r'<PREP> <DET>? (<NOUN>+<ADP>)* <NOUN>+'
        Yields:
            ``spacy.Span``: the next span of consecutive tokens from ``doc`` whose
            parts-of-speech match ``pattern``, in order of apperance
        """
        # standardize and transform the regular expression pattern...
        pattern = re.sub(r'\s', '', pattern)
        pattern = re.sub(r'<([A-Z]+)\|([A-Z]+)>', r'( (\1|\2))', pattern)
        pattern = re.sub(r'<([A-Z]+)>', r'( \1)', pattern)

        tags = ' ' + ' '.join(tok.pos_ for tok in doc)

        for m in re.finditer(pattern, tags):
            yield doc[tags[0:m.start()].count(' '):tags[0:m.end()].count(' ')]


    def subject_verb_object_triples(doc):
        """
        Extract an ordered sequence of subject-verb-object (SVO) triples from a
        spacy-parsed doc. Note that this only works for SVO languages.
        Args:
            doc (``textacy.Doc`` or ``spacy.Doc`` or ``spacy.Span``)
        Yields:
            Tuple[``spacy.Span``, ``spacy.Span``, ``spacy.Span``]: The next 3-tuple
            of spans from ``doc`` representing a (subject, verb, object) triple,
            in order of appearance.
        """
        # TODO: What to do about questions, where it may be VSO instead of SVO?
        # TODO: What about non-adjacent verb negations?
        # TODO: What about object (noun) negations?
        if isinstance(doc, SpacySpan):
            sents = [doc]
        else:  # textacy.Doc or spacy.Doc
            sents = doc.sents

        for sent in sents:
            start_i = sent[0].i

            verbs = spacy_utils.get_main_verbs_of_sent(sent)
            for verb in verbs:
                subjs = spacy_utils.get_subjects_of_verb(verb)
                if not subjs:
                    continue
                objs = spacy_utils.get_objects_of_verb(verb)
                if not objs:
                    continue

                # add adjacent auxiliaries to verbs, for context
                # and add compounds to compound nouns
                verb_span = spacy_utils.get_span_for_verb_auxiliaries(verb)
                verb = sent[verb_span[0] - start_i: verb_span[1] - start_i + 1]
                for subj in subjs:
                    subj = sent[spacy_utils.get_span_for_compound_noun(subj)[0] - start_i: subj.i - start_i + 1]
                    for obj in objs:
                        if obj.pos == NOUN:
                            span = spacy_utils.get_span_for_compound_noun(obj)
                        elif obj.pos == VERB:
                            span = spacy_utils.get_span_for_verb_auxiliaries(obj)
                        else:
                            span = (obj.i, obj.i)
                        obj = sent[span[0] - start_i: span[1] - start_i + 1]

                        yield (subj, verb, obj)

    def get_span_for_compound_noun(noun):
        """
        Return document indexes spanning all (adjacent) tokens
        in a compound noun.
        """
        min_i = noun.i - sum(1 for _ in takewhile(lambda x: x.dep_ in ['compound','amod'],
                                                  reversed(list(noun.lefts))))
        return (min_i, noun.i)

    def get_span_for_verb_auxiliaries(verb):
        """
        Return document indexes spanning all (adjacent) tokens
        around a verb that are auxiliary verbs or negations.
        """
        min_i = verb.i - sum(1 for _ in takewhile(lambda x: x.dep_ in AUX_DEPS1,
                                                  reversed(list(verb.lefts))))
        max_i = verb.i + sum(1 for _ in takewhile(lambda x: x.dep_ in AUX_DEPS1,
                                                  verb.rights))
        return (min_i, max_i)


    def flatten( alist ):
        newlist = []
        for item in alist:
            if isinstance(item, list):
                newlist = newlist + flatten(item)
            else:
                newlist.append(item)
        return newlist

    def _get_conjunctsv(tok):
        """
        Return conjunct dependents of the leftmost conjunct in a coordinated phrase,
        e.g. "Burton, [Dan], and [Josh] ...".
        """
        return [right for right in tok.rights
                if right.dep_ == 'conj']
               #if right.dep_ == 'conj' and right.pos == VERB]

    def _get_conjunctsn(tok):
        """
        Return conjunct dependents of the leftmost conjunct in a coordinated phrase,
        e.g. "Burton, [Dan], and [Josh] ...".
        """
        return [right for right in tok.rights
                if right.dep_ == 'conj']
               #if right.dep_ == 'conj' and right.pos == NOUN]

    def get_subjects_of_sent(sent):
        """Return the main subjects in a sentence."""
        subjs = [tok for tok in sent
                 if tok.dep_ in SUBJ_DEPS]
        # get additional conjunct subjects
        subjs.extend(tok for subj in subjs for tok in _get_conjunctsn(subj))
        return subjs

    def get_main_nouns_of_sent(doc):
        """Return the main nouns in a sentence."""
        return [tok for tok in doc
                if tok.pos == NOUN ]

    def getstringlist(listspacy):
        """ Return list with list elements converted from spacy tokens to strings """
        liststring = []
        for x in listspacy:
            liststring.append(x.string)
        return liststring

    def rep_nouns_verbs_with_np_vp(nvlist,npvplist):  
        """ Returns a list with noun phrases from given nouns in a given sentence """
        npvpslist = []
        for sub in nvlist:
                x = [s for s in npvplist if sub in s]
                npvpslist.append(x)
        return npvpslist


    # def _get_prepositions(tok):
    #     """ Return prepositions of the token listed in a doc """
    #     return [right for right in tok.rights
    #             if right.dep_ in ['prep','agent','acomp','xcomp']]


    def _get_prepositions(tok):
        """ Return prepositions of the token listed in a doc """
        preps = [right for right in tok.rights
                    if right.dep_ in ['prep','agent','acomp','xcomp']]
        preps.extend(left for left in tok.lefts
                                 if left.dep_ in ['prep'])
        return preps

    def getprepobjects(tok):
        objs = [right for right in tok.rights
                if right.dep_ == 'pobj']
        # get open clausal complements (xcomp)
        objs.extend(tok for tok in tok.rights
                    if tok.dep_ == 'xcomp')
        objs.extend(tok for obj in objs for tok in _get_conjunctsv(obj))
        return objs

    def prepositionslist(nounverblist):
        nprep = []
        for n1 in nounverblist:
            cj = _get_prepositions(n1)
            nprep.append(cj)
        return nprep

    def get_conj_list_verbnoun(nvlist):
        conjlist = []
        for n1 in nvlist:
            i = 0
            cj = _get_conjunctsn(n1)    
            if i == 0: 
                i = 1
               #conjlist.append(n1)
                conjlist.append(cj)
            else:
                conjlist.append(cj)
        return conjlist         

    """
    xall contains all conjunctions of verbs in a given sentence
    this function will return all the conjunctions that were linked to any  given verb
    """
    def getverbconj(y,xall):
        xs = []
        for xindex in xall:
            if y in xindex:
                xs.append(xindex)
        xs = flatten(xs)
        xs = list(set(xs))
        return xs


    """
    yall contains all the objects linked to given prepositions
    xall contains all conjunctions list
    this function will return all the conjunctions that were linked to any noun/verb
    """
    def getobjconj(yall,xall):
        xs = []
        for y in yall:
            for x in xall:
                if y in x:
                    xs.append(x)
        xs = flatten(xs)
        xs = list(set(xs))
        return xs

    """
    yall contains all the objects linked to given prepositions
    xall contains all conjunctions list
    this function will return all the conjunctions that were linked to any noun/verb
    """
    def getobjconjtest(yone,xall):
        xs = []
        #for y in yall:
        for x in xall:
                if yone in x:
                    xs.append(x)
        xs = flatten(xs)
        xs = list(set(xs))

        return xs

    def mergeverbconj(l):

        """
            Algorithm Logic:
            1. take first set A from list
            2. for each other set B in the list do if B has common element(s) with A join B into A; remove B from list
            3. repeat 2. until no more overlap with A
            4. put A into outpup
            5. repeat 1. with rest of list 
        """

        out = []
        while len(l)>0:
            first, *rest = l
            first = set(first)

            lf = -1
            while len(first)>lf:
                lf = len(first)

                rest2 = []
                for r in rest:
                    if len(first.intersection(set(r)))>0:
                        first |= set(r)
                    else:
                        rest2.append(r)     
                rest = rest2

            out.append(first)
            l = rest

        return out

    def get_all_noun_conjunctions(doc):
        """
            Algorithm Logic
                1. Get all nouns from the given sentence outputs a list
                2. for every noun get the conjunctions linked to it this ouputs a list
                3. zip two list to get conjunctions associated to each noun in a list
                4. flatten the tuple elements for every element in the zipped list
                5. merge nounconjunctions which were linked to each other in the set
        """
        nouns = get_main_nouns_of_sent(doc)
        nounconj = get_conj_list_verbnoun(nouns)
        nounconj =list(zip(nouns,nounconj))
        nounconjf = []
        nounconjf =[(a, *rest) for a, rest in nounconj]
        nounconjs = mergeverbconj(nounconjf)
        nounconjll = []
        for x in nounconjs:
             nounconjll.append(list(x))
        return nounconjll

    def get_all_verb_conjunctions(doc):
        """
        Algorithm Logic
        1. Get all verbs from the given sentence outputs a list
        2. for every verb get the conjunctions linked to it this ouputs a list
        3. zip two list to get conjunctions associated to each verb
        4. flatten the tuple elements for every element in the zipped list
        5. merge verbconjunctions which were linked to each other in the set
        """
        verbs =[]
        verbs = spacy_utils.get_main_verbs_of_sent(doc)
        verbconj = []
        verbconj = get_conj_list_verbnoun(verbs)
        verbconjl = []
        verbconjl =list(zip(verbs,verbconj))
        verbconjf = []
        verbconjf =[(a, *rest) for a, rest in verbconjl]
        verbconjs = mergeverbconj(verbconjf)
        verbconjll = []
        for x in verbconjs:
            verbconjll.append(list(x))           
        return verbconjll

    def get_objects_of_verb(verb):
        """
        Return all objects of a verb according to the dependency parse,
        including open clausal complements.
        """
        objs = [tok for tok in verb.rights
                if tok.dep_ in OBJ_DEPS1]    
        objs.extend(tok for tok in verb.lefts
                if tok.dep_ in OBJ_DEPS1)
        # get open clausal complements (xcomp)
        objs.extend(tok for tok in verb.rights
                    if tok.dep_ == 'xcomp')
        # get additional conjunct objects
        objs.extend(tok for obj in objs for tok in spacy_utils._get_conjuncts(obj))
        return objs

    def get_all_verb_advcl(doc):
        """
        Algorithm Logic
        1. Get all verbs from the given sentence outputs a list
        2. for every verb get the conjunctions linked to it this ouputs a list
        3. zip two list to get conjunctions associated to each verb
        4. flatten the tuple elements for every element in the zipped list
        5. merge verbconjunctions which were linked to each other in the set
        """
        verbs =[]
        verbs = spacy_utils.get_main_verbs_of_sent(doc)
        advclverbs = []
        for verb in verbs:
            advclverbs.append([tok for tok in verb.rights
                                    if tok.dep_  in ['advcl']])
        verbconjl = []
        verbconjl =list(zip(verbs,advclverbs))
        verbconjf = []
        verbconjf =[(a, *rest) for a, rest in verbconjl]
        #verbconjs = mergeverbconj(verbconjf)
        verbconjll = []
        for x in verbconjf:
             verbconjll.append(list(x))
        return verbconjll

    def get_all_verb_acl(doc):
        nouns = get_main_nouns_of_sent(doc)
        acllist = []
        for noun in nouns:
            acllist.append([tok for tok in noun.rights
                        if tok.dep_  in ['acl']])
        verbconjl = []
        verbconjl =list(zip(nouns,acllist))
        verbconjf = []
        verbconjf =[(a, *rest) for a, rest in verbconjl]
        verbconjll = []
        for x in verbconjf:
            verbconjll.append(list(x))
        return verbconjll


    def get_all_verb_xcomp(doc):
        """
        Algorithm Logic
        1. Get all verbs from the given sentence outputs a list
        2. for every verb get the xcomp linked to it this ouputs a list
        3. zip two list to get xcomp associated to each verb
        4. flatten the tuple elements for every element in the zipped list
        5. merge verbconjunctions which were linked to each other in the set
        """
        verbs =[]
        verbs = spacy_utils.get_main_verbs_of_sent(doc)
        xcompverbs = []
        for verb in verbs:
            xcompverbs.append([tok for tok in verb.rights
                                    if tok.dep_  in ['xcomp']])
        verbconjl = []
        verbconjl =list(zip(verbs,xcompverbs))
        verbconjf = []
        verbconjf =[(a, *rest) for a, rest in verbconjl]
        #verbconjs = mergeverbconj(verbconjf)
        verbconjll = []
        for x in verbconjf:
             verbconjll.append(list(x))
        return verbconjll

    def getsubjects_ofthe_verb(verb,sent):
            start_i = sent[0].i

            acllist   = get_all_verb_acl(sent)
            advclverbs= get_all_verb_advcl(sent)
            xcompverbs= get_all_verb_xcomp(sent)
            subjsa = get_subjects_of_sent(sent)
            nounconjl = get_all_noun_conjunctions(sent)                   # get all noun conjunctions in the given document
            verbconjl = get_all_verb_conjunctions(sent)                   # get all verb conjunctions in the given document

            subjs = spacy_utils.get_subjects_of_verb(verb) 
            verbsy = []

            # if direct subjects doesn't exist for verb , get subject from conjunction verbs to it
            conjverb = getobjconjtest(verb,verbconjl)
            conjnoun = getobjconjtest(verb,nounconjl)
            if len(conjverb) > 0:
                verbsy.extend(conjverb)
            if len(conjnoun) > 0:
                verbsy.extend(conjnoun)

            if len(subjs) == 0:
                for cverbs in verbsy:
                    subjs.extend(spacy_utils.get_subjects_of_verb(cverbs))

            # check for open clausal compliment exist (xcomp) for verb
            if len(subjs) == 0:
                xcompx = getobjconjtest(verb,xcompverbs)
                if len(xcompx) > 1:
                    for xcompxx in xcompx:
                        subjs.extend(spacy_utils.get_subjects_of_verb(xcompxx))

            # if direct subjects and conjunctional subjects doesn't exist for verb , get subject from adverbial clauses
            if len(subjs) == 0:
                advclx = getobjconjtest(verb,advclverbs)
                if len(advclx) > 1:
                    for advclxx in advclx:
                        subjs.extend(spacy_utils.get_subjects_of_verb(advclxx))

            # if direct subjects, conjunctional verb subjects, adverbial clause subjects doesn't exist then check any acl nouns
            if len(subjs) == 0:
                aclx = getobjconjtest(verb,acllist)
                if len(aclx) > 1:
                    subjsx = []
                    subjsx.extend([x for x in aclx if x != verb])
                    if len(subjsx) > 0:                   
                        subjsy = []
                        for x in subjsx:
                            subjsy.extend(getobjconjtest(x,nounconjl))
                        if len(subjsy) > 0:
                            subjs.extend(subjsy)
                    if len(subjs) == 0:
                        subjs.extend(subjsx)

            # if no subject exist get the default first subject of the sentence  
            if len(subjs) == 0:
                if len(subjsa) > 0:
                    subjs.append(subjsa[0])  
                    subjs.extend(getobjconjtest(subjsa[0],nounconjl))

            subjs11 = []
            subjsall = []
            # replace subjects with prepositions and its objects 
            for subjs1 in subjs:
                preps = _get_prepositions(subjs1)
                if len(preps) > 0: 
                    for preps1 in preps:
                        objspo = getprepobjects(preps1)
                        if len(objspo) > 0:
                            for objspo1 in objspo:
                                subjs11 = getobjconjtest(objspo1,nounconjl)
                                if len(subjs11) > 0:
                                    for subjs111 in subjs11:
                                        subjs1x = replaceobj_withcompoundobj(subjs1,sent)
                                        subjs111x = replaceobj_withcompoundobj(subjs111,sent)
                                        subjsall.extend([subjs1x.string + preps1.string + subjs111x.string])
                else:
                    subjs1x = replaceobj_withcompoundobj(subjs1,sent)
                    subjsall.extend([subjs1x.string])

            subjs = subjsall

            return subjs


    def replace_objs_with_compound_objs(objs,sent):
        start_i = sent[0].i

        objs1 = []
        while len(objs1) > 0: objs1.pop()
        for obj in objs:
                if obj.pos == NOUN:
                    span = get_span_for_compound_noun(obj)
                elif obj.pos == VERB:
                    span = get_span_for_verb_auxiliaries(obj)
                else:
                    span = (obj.i, obj.i)
                obj = sent[span[0] - start_i: span[1] - start_i + 1]

                objs1.append(obj)
        return objs1


    # def replace_verbs_with_auxillary_verbs(verbs,sent):
    #     start_i = sent[0].i

    #     verbsl = []
    #     while len(verbsl) > 0: verbsl.pop()
    #     for verb in verbs:
    #         verb_span = get_span_for_verb_auxiliaries(verb)
    #         verb = sent[verb_span[0] - start_i: verb_span[1] - start_i + 1]

    #         verbsl.append(verb)
    #     return verbsl

    def replace_verbs_with_auxillary_verbs(verb,verbs,sent):
        start_i = sent[0].i

        verbs.remove(verb)
        verbsl = []
        while len(verbsl) > 0: verbsl.pop()

        if len(verbs) > 0:
            for verb in verbs:
                verb_span = get_span_for_verb_auxiliaries(verb)
                verbq = sent[verb_span[0] - start_i: verb_span[1] - start_i + 1]

                verbsl.append(verbq.string)

        xcompverb = getxcomp_ofthe_verb(verb,sent)   # get xcomp (open clausal compliment) of verb if exist
        if len(xcompverb) > 0: 
            verbx = xcompverb[0]
            verbsl.insert(0,verbx)
        else:
            verb_span = get_span_for_verb_auxiliaries(verb)
            verbq = sent[verb_span[0] - start_i: verb_span[1] - start_i + 1]
            verbsl.insert(0,verbq.string)

        return verbsl

    def getxcomp_ofthe_verb(verb,sent):
        start_i = sent[0].i
        xcompxx=[]
        xcompy = []
        xcompverbs= get_all_verb_xcomp(sent)
        xcompx = getobjconjtest(verb,xcompverbs)
        xcompy = [x for x in xcompx if x != verb]
        m1 = [tok for tok in verb.rights
                            if tok.dep_  in ['xcomp']]
        if len(m1) == 0 and len(xcompy) > 0:
            verbxt = replaceverb_withauxillaryverb(xcompy[0],sent)
            verbx  = replaceverb_withauxillaryverb(verb,sent)
            verbx = verbxt + verbx
            xcompxx.insert(0,verbx)

        return xcompxx

    def replaceverb_withauxillaryverb(verb,sent):
        start_i = sent[0].i

        verb_span = get_span_for_verb_auxiliaries(verb)
        verb = sent[verb_span[0] - start_i: verb_span[1] - start_i + 1]
        return verb.string

    def replaceobj_withcompoundobj(obj,sent):
        start_i = sent[0].i

        if obj.pos == NOUN:
            span = get_span_for_compound_noun(obj)
        elif obj.pos == VERB:
            span = get_span_for_verb_auxiliaries(obj)
        else:
            span = (obj.i, obj.i)
        obj = sent[span[0] - start_i: span[1] - start_i + 1]


        return obj  

    def verbswithdobjs(verb,cverbsy,sent):

        verbswithdobj = []
        verbs2 = spacy_utils.get_main_verbs_of_sent(sent)
        for verb21 in verbs2:
            objs =  get_objects_of_verb(verb21)
            if len(objs) > 0:
                verbswithdobj.insert(0,verb21)


        v1 = [x for x in cverbsy if x != verb]
        for v11 in v1:
            if v11 in verbswithdobj:
                v1.remove(v11)
        v1.insert(0,verb)
        return v1


    # verb---object---prep1---pobj1/pobj11/pobj12
    #        or    ---prep2---pobj2/pobj21/pobj22
    #        (acomp)

    # verb---object---prep1---pobj1---prep2---pobj21/pobj22
    #        or    ---prep3---pobj3---prep4---pobj41/pobj42
    #        (acomp)

    def subject_verb_object_pattern1(doc):

        if isinstance(doc, SpacySpan):
            sents = [doc]
        else:  # textacy.Doc or spacy.Doc
            sents = doc.sents

        for sent in sents:
            start_i = sent[0].i

            acllist   = get_all_verb_acl(sent)
            advclverbs= get_all_verb_advcl(sent)
            nounconjl = get_all_noun_conjunctions(sent)                   # get all noun conjunctions in the given document
            verbconjl = get_all_verb_conjunctions(sent)                   # get all verb conjunctions in the given document
            subjsa = get_subjects_of_sent(sent)                          # get all subjects in a sentence

            verbs = []
            while len(verbs) > 0 : verbs.pop()
            verbs = spacy_utils.get_main_verbs_of_sent(sent)
            """
            Algorithm logic to extract non-verbs treated as verbs
            1. Get all the nouns from the sentence
            2. for every noun check if either subject or object exists to it 
            3. If exists then add noun to the verbs list
            """


            for verb in verbs:
                """
                Algorithm logic to extract subjects of the verb

                1. Check for any subjects exists for the given verb
                2. If no subjects exists then get the main subject of the sentence as the subject
                3. check if any conjunctions exists to the subject, if conjunctions exist add conjunctions to the subject
                4. check if any prepositions exists for the subjects, if exists then extract prepositional subjects to the sentence
                       check for any conjunctions exists for prepositional objects, if present then add all the subjects to subject list
                """

                subjs = getsubjects_ofthe_verb(verb,sent)

                """
                   Algorithm logic to extract objects of the verb

                   1. Check for any objects exists for the given verb
                   2. check if any conjunctions exists to the object, if conjunctions exist add conjunctions to the object list
                   3. check if any prepositions exists for the objects, if exists then extract prepositional objects to the sentence
                      check for any conjunctions exists for prepositional objects, if present then add all the objects to object list
                """               
                xcompverb = getxcomp_ofthe_verb(verb,sent)   # get xcomp (open clausal compliment) of verb if exist
                objs = spacy_utils.get_objects_of_verb(verb)

                objacomp = [tok for tok in verb.rights
                                    if tok.dep_  in ['acomp']]     # check if any acomp objects exist for verb
                if len(objacomp) > 0:
                    objs.extend(objacomp) 

                objsc = getobjconj(objs,nounconjl)                     # get all object conjunctions to object list
                if len(objsc) > 1:
                    objs.extend(objsc)                                     # add conjunctions to object list


                # Check if verb objects are having any prepositions and prepositional objects and its conjunctions
                objsy = []
                verbsy = []
                objspo = []
                objs9 = []
                cverbsy = []
                verbstemp = []

                for objsc1 in objs:
                    preps = _get_prepositions(objsc1)                    # check for any prepositions exist for that object
                    if len(preps) > 0:                                 # check atleast one preposition found
                        for prepsx in preps:
                            while len(objspo)>0 : objspo.pop()
                            objspo = getprepobjects(prepsx)                # get pcomp object exists for preposition
                            if len(objspo) > 0:                              # check for atleast one prepositional object found
                                for objspox in objspo:
                                    preps1 = _get_prepositions(objspox)
                                    if len(preps1) == 0:                                 # check atleast one preposition found
                                        while len(objs9) > 0 : objs9.pop()
                                        objs9.extend(objspo)
                                        objs1 = getobjconjtest(objspox,nounconjl) 
                                        if len(objs1) > 1:   
                                            objs9.extend(objs1)
                                            objs9 = list(set(objs9))

                                        objs9 = replace_objs_with_compound_objs(objs9,sent)
                                        """Replacing verbs with conjunction verbs
                                        """ 
                                        while len(cverbsy) > 0 : cverbsy.pop()
                                        while len(verbstemp) > 0 : verbstemp.pop()
                                        conjverb = getobjconjtest(verb,verbconjl)
                                        conjnoun = getobjconjtest(verb,nounconjl)
                                        if len(conjverb) > 0:
                                            cverbsy.extend(conjverb)
                                        if len(conjnoun) > 0:
                                            cverbsy.extend(conjnoun)
                                        cverbsy = verbswithdobjs(verb,cverbsy,sent)  # remove conjunction verbs if they have direct objects exists

                                        cverbsy = replace_verbs_with_auxillary_verbs(verb,cverbsy,sent)
                                        objsc1conj = getobjconjtest(objsc1,nounconjl)

                                        objsc1conj = replace_objs_with_compound_objs(objsc1conj,sent)
                                        objsc1x = replaceobj_withcompoundobj(objsc1,sent)
                                        verbx = replaceverb_withauxillaryverb(verb,sent)
                                        if len(xcompverb) > 0: verbx = xcompverb[0]


                                        if len(cverbsy) > 1:
                                            if len(objsc1conj) > 1:
                                                for cverbsy1 in cverbsy:
                                                    for objsc1conj1 in objsc1conj:
                                                        verbstemp.extend([cverbsy1 + objsc1conj1.string + prepsx.string])
                                            else:
                                                for cverbsy1 in cverbsy:
                                                    verbstemp.extend([cverbsy1 + objsc1x.string + prepsx.string])
                                        else:
                                            if len(objsc1conj) > 1:
                                                for objsc1conj1 in objsc1conj:
                                                    verbstemp.extend([verbx + objsc1conj1.string + prepsx.string])
                                            else:
                                                verbstemp = [verbx + objsc1x.string + prepsx.string]

                                        verbsy = verbstemp
                                        """Print triplets with extracted subjects, verbs and objects
                                        """
                                        if (len(subjs) > 0) and (len(objs9)) > 0  and (len(verbsy) > 0):
                                            for verb1 in verbsy:
                                                for subj in subjs:
                                                    for obj in objs9:
                                                        yield (subj, verb1, obj)
                                    else: 
                                        for prepsx1 in preps1:
                                            objspo1 = getprepobjects(prepsx1)                # get pcomp object exists for preposition
                                            if len(objspo1) > 0:                              # check for atleast one prepositional object found
                                                for objspox1 in objspo1:
                                                    objs9 = objspo1
                                                    objs11 = getobjconjtest(objspox1,nounconjl)          # get all conjunctions to given nounlist 
                                                    if len(objs11) > 1:                         # if any conjunctions found for the object
                                                        objs9.extend(objs11)
                                                        objs9 = list(set(objs9))
                                                        objspo1 = list(set(objspo1))
                                                    objs9 = list(set(objs9))
                                                    objs9 = replace_objs_with_compound_objs(objs9,sent)

                                                    while len(objs11) > 0 : objs11.pop()

                                                    """Replacing verbs with conjunction verbs
                                                    """
                                                    while len(cverbsy) > 0 : cverbsy.pop()
                                                    while len(verbstemp) > 0 : verbstemp.pop()
                                                    conjverb = getobjconjtest(verb,verbconjl)
                                                    conjnoun = getobjconjtest(verb,nounconjl)
                                                    if len(conjverb) > 0:
                                                        cverbsy.extend(conjverb)
                                                    if len(conjnoun) > 0:
                                                        cverbsy.extend(conjnoun)

                                                    cverbsy = verbswithdobjs(verb,cverbsy,sent)

                                                    cverbsy = replace_verbs_with_auxillary_verbs(verb,cverbsy,sent)
                                                    objsc1x = replaceobj_withcompoundobj(objsc1,sent)
                                                    objspoxx = replaceobj_withcompoundobj(objspox,sent)
                                                    verbx = replaceverb_withauxillaryverb(verb,sent)
                                                    if len(xcompverb) > 0: verbx = xcompverb[0]

                                                    if len(cverbsy) > 1:
                                                        for cverbsy1 in cverbsy:
                                                            verbstemp.extend([cverbsy1 + objsc1x.string + prepsx.string + objspoxx.string + prepsx1.string])
                                                    else:
                                                            verbstemp = [verbx + objsc1x.string + prepsx.string + objspoxx.string + prepsx1.string]

                                                    verbsy = verbstemp


                                                    #verbstemp = [verb.string + objsc1.string + prepsx.string + objspox.string + prepsx1.string]
                                                    #verbsy = verbstemp
                                                    """Print triplets with extracted subjects, verbs and objects
                                                    """
                                                    if (len(subjs) > 0) and (len(objs9)) > 0  and (len(verbsy) > 0):
                                                        for verb1 in verbsy:
                                                            for subj in subjs:
                                                                for obj1 in objs9:
                                                                    yield (subj, verb1, obj1)                                    
                                                while len(objspo1) > 0: objspo1.pop()
                                        while len(preps1) > 0 : preps1.pop()
                                while len(objspo) > 0 : objspo.pop()
                        while len(preps)>0 : preps.pop()

    # verbs---prep1---pobj1/pobj11/pobj12
    #      ---prep2---pobj2/pobj21/pobj22

    #verbs---prep1---pobj1---prep2---pobj21/pobj22
    #                     ---prep3---pobj31/pobj32

    def subject_verb_object_pattern2(doc):

        if isinstance(doc, SpacySpan):
            sents = [doc]
        else:  # textacy.Doc or spacy.Doc
            sents = doc.sents

        for sent in sents:
            start_i = sent[0].i

            nounconjl = get_all_noun_conjunctions(sent)                   # get all noun conjunctions in the given document
            verbconjl = get_all_verb_conjunctions(sent)                   # get all verb conjunctions in the given document
            subjsa = get_subjects_of_sent(sent)                          # get all subjects in a sentence

            verbs = []
            while len(verbs) > 0 : verbs.pop()
            verbs = spacy_utils.get_main_verbs_of_sent(sent)
            """
            Algorithm logic to extract non-verbs treated as verbs
            1. Get all the nouns from the sentence
            2. for every noun check if either subject or object exists to it 
            3. If exists then add noun to the verbs list
            """


            for verb in verbs:
                """
                Algorithm logic to extract subjects of the verb

                1. Check for any subjects exists for the given verb
                2. If no subjects exists then get the main subject of the sentence as the subject
                3. check if any conjunctions exists to the subject, if conjunctions exist add conjunctions to the subject
                4. check if any prepositions exists for the subjects, if exists then extract prepositional subjects to the sentence
                       check for any conjunctions exists for prepositional objects, if present then add all the subjects to subject list
                """

                subjs = getsubjects_ofthe_verb(verb,sent)
                """
                   Algorithm logic to extract objects of the verb

                   1. Check for any objects exists for the given verb
                   2. check if any conjunctions exists to the object, if conjunctions exist add conjunctions to the object list
                   3. check if any prepositions exists for the objects, if exists then extract prepositional objects to the sentence
                      check for any conjunctions exists for prepositional objects, if present then add all the objects to object list
                """               
                xcompverb = getxcomp_ofthe_verb(verb,sent)   # get xcomp (open clausal compliment) of verb if exist
                objs = []
                objs = spacy_utils.get_objects_of_verb(verb)
                objsy = []
                objsy1 = []
                verbsy = []
                objspo = []
                objs9 = []    
                verbstemp = []
                cverbsy = []
                # check if verb is having prepositions and prepositional objects and its conjunctions                

                preps = _get_prepositions(verb)                    # check for any prepositions exist for that object
                if len(preps) > 0 and len(objs) == 0 :                                 # check atleast one preposition found
                    for prepsx in preps:
                        while len(objspo) > 0: objspo.pop() 
                        objspo = getprepobjects(prepsx)                # get pcomp object exists for preposition
                        if len(objspo) > 0:                              # check for atleast one prepositional object found
                            for objspox in objspo:
                                preps1 = _get_prepositions(objspox)
                                if len(preps1) == 0:                                 # check atleast one preposition found
                                    while len(objsy) > 0: objsy.pop()
                                    objsy.extend(objspo)
                                    objs1 = getobjconjtest(objspox,nounconjl) 
                                    if len(objs1) > 1:   
                                        objsy.extend(objs1)  
                                        objsy = list(set(objsy))
                                    objsy = replace_objs_with_compound_objs(objsy,sent)
                                    """ Replace verbs with conjunction verbs
                                    """
                                    while len(cverbsy) > 0 : cverbsy.pop()
                                    while len(verbstemp) > 0 : verbstemp.pop()
                                    conjverb = getobjconjtest(verb,verbconjl)
                                    conjnoun = getobjconjtest(verb,nounconjl)
                                    if len(conjverb) > 0:
                                        cverbsy.extend(conjverb)
                                    if len(conjnoun) > 0:
                                        cverbsy.extend(conjnoun)

                                    cverbsy = verbswithdobjs(verb,cverbsy,sent)

                                    cverbsy = replace_verbs_with_auxillary_verbs(verb,cverbsy,sent)
                                    verbx = replaceverb_withauxillaryverb(verb,sent)
                                    if len(xcompverb) > 0: verbx = xcompverb[0]

                                    if len(cverbsy) > 1:
                                        for cverbsy1 in cverbsy:
                                            verbstemp.extend([cverbsy1 + prepsx.string])
                                    else:
                                        verbstemp = [verbx + prepsx.string]
                                    verbsy = verbstemp        
                                    #verbstemp = [verb.string + prepsx.string]
                                    #verbsy = verbstemp 
                                    if (len(subjs) > 0) and (len(objsy)) > 0  and (len(verbsy) > 0):
                                        for verb1 in verbsy:
                                            for subj in subjs:
                                                for obj in objsy:
                                                    yield (subj, verb1, obj)

                                else:    

                                    for prepsx1 in preps1:
                                        objspo1 = getprepobjects(prepsx1)                # get pcomp object exists for preposition
                                        if len(objspo1) > 0:                              # check for atleast one prepositional object found
                                            for objspox1 in objspo1:
                                                while len(objsy1) > 0: objsy1.pop()
                                                objsy1.extend(objspo1)
                                                objs11 = getobjconjtest(objspox1,nounconjl)          # get all conjunctions to given nounlist 
                                                if len(objs11) > 1:                         # if any conjunctions found for the object
                                                    objsy1.extend(objs11)                           # replace retrieved objects to objs list 
                                                    objsy1 = list(set(objsy))
                                                objsy1 = replace_objs_with_compound_objs(objsy1,sent)

                                                """ Replace verbs with conjunction verbs
                                                """
                                                while len(cverbsy) > 0 : cverbsy.pop()
                                                while len(verbstemp) > 0 : verbstemp.pop()
                                                conjverb = getobjconjtest(verb,verbconjl)
                                                conjnoun = getobjconjtest(verb,nounconjl)
                                                if len(conjverb) > 0:
                                                    cverbsy.extend(conjverb)
                                                if len(conjnoun) > 0:
                                                    cverbsy.extend(conjnoun)
                                                cverbsy = verbswithdobjs(verb,cverbsy,sent)

                                                cverbsy = replace_verbs_with_auxillary_verbs(verb,cverbsy,sent)
                                                objspoxx = replaceobj_withcompoundobj(objspox,sent)
                                                verbx    = replaceverb_withauxillaryverb(verb,sent)
                                                if len(xcompverb) > 0: verbx = xcompverb[0]

                                                if len(cverbsy) > 1:
                                                    for cverbsy1 in cverbsy:
                                                        verbstemp.extend([cverbsy1 + prepsx.string + objspoxx.string + prepsx1.string])
                                                else:
                                                    verbstemp = [verbx + prepsx.string + objspoxx.string + prepsx1.string]
                                                verbsy = verbstemp        



                                                #verbstemp = [verb.string + prepsx.string + objspox.string + prepsx1.string]
                                                #verbsy = verbstemp
                                                if (len(subjs) > 0) and (len(objsy1)) > 0  and (len(verbsy) > 0):
                                                    for verb1 in verbsy:
                                                        for subj in subjs:
                                                            for obj1 in objsy1:
                                                                yield (subj, verb1, obj1)   
                                            while len(objspo1) > 0: objspo1.pop()                    
                                    while len(preps1) > 0 : preps1.pop()
                            while len(objspo) > 0 : objspo.pop()
                    while len(preps)>0 : preps.pop()            


    #verb---prep1(verb)---prep2---pobj11/pobj12

    def subject_verb_object_pattern3(doc):

        if isinstance(doc, SpacySpan):
            sents = [doc]
        else:  # textacy.Doc or spacy.Doc
            sents = doc.sents

        for sent in sents:
            start_i = sent[0].i

            nounconjl = get_all_noun_conjunctions(sent)                   # get all noun conjunctions in the given document
            verbconjl = get_all_verb_conjunctions(sent)                   # get all verb conjunctions in the given document
            subjsa = get_subjects_of_sent(sent)                          # get all subjects in a sentence

            verbs = []
            while len(verbs) > 0 : verbs.pop()
            verbs = spacy_utils.get_main_verbs_of_sent(sent)
            """
            Algorithm logic to extract non-verbs treated as verbs
            1. Get all the nouns from the sentence
            2. for every noun check if either subject or object exists to it 
            3. If exists then add noun to the verbs list
            """


            for verb in verbs:
                """
                Algorithm logic to extract subjects of the verb

                1. Check for any subjects exists for the given verb
                2. If no subjects exists then get the main subject of the sentence as the subject
                3. check if any conjunctions exists to the subject, if conjunctions exist add conjunctions to the subject
                4. check if any prepositions exists for the subjects, if exists then extract prepositional subjects to the sentence
                       check for any conjunctions exists for prepositional objects, if present then add all the subjects to subject list
                """

                subjs = getsubjects_ofthe_verb(verb,sent)

                """
                   Algorithm logic to extract objects of the verb

                   1. Check for any objects exists for the given verb
                   2. check if any conjunctions exists to the object, if conjunctions exist add conjunctions to the object list
                   3. check if any prepositions exists for the objects, if exists then extract prepositional objects to the sentence
                      check for any conjunctions exists for prepositional objects, if present then add all the objects to object list
                """               
                xcompverb = getxcomp_ofthe_verb(verb,sent)   # get xcomp (open clausal compliment) of verb if exist
                objs = []
                objsy = []
                verbsy = []
                objs11 = []
                objspo1 = []
                prepsx1 = []
                preps = []
                verbstemp = []
                cverbsy = []

                # check if verb is having multiple prepositions which are verbs, which again have prepositions and prepositional objects and its conjunctions
                verbsy = []
                preps = _get_prepositions(verb)                    # check for any prepositions exist for that object
                if len(preps) > 0:                                 # check atleast one preposition found
                    for prepsx in preps:
                        if prepsx.pos == VERB:
                            prepsx1 = _get_prepositions(prepsx)                    # check for any prepositions exist for that object
                            objs    =  get_objects_of_verb(prepsx)
                            if len(prepsx1) > 0 and len(objs) == 0:                                 # check atleast one preposition found
                                for prepsx11 in prepsx1:
                                    objspo1 = getprepobjects(prepsx11)    
                                    if len(objspo1) > 0:                              # check for atleast one prepositional object found
                                        for objspox1 in objspo1:
                                            while len(objsy) > 0 : objsy.pop()
                                            objsy.extend(objspo1)
                                            objs11 = getobjconjtest(objspox1,nounconjl)          # get all conjunctions to given nounlist 
                                            if len(objs11) > 0:                         # if any conjunctions found for the object
                                                objsy.extend(objs11)                           # replace retrieved objects to objs list 
                                                objsy = list(set(objsy))
                                            objsy = replace_objs_with_compound_objs(objsy,sent)

                                            """ Replace verbs with conjunction verbs
                                            """
                                            while len(cverbsy) > 0 : cverbsy.pop()
                                            while len(verbstemp) > 0 : verbstemp.pop()
                                            conjverb = getobjconjtest(verb,verbconjl)
                                            conjnoun = getobjconjtest(verb,nounconjl)
                                            if len(conjverb) > 0:
                                                cverbsy.extend(conjverb)
                                            if len(conjnoun) > 0:
                                                cverbsy.extend(conjnoun)

                                            cverbsy = verbswithdobjs(verb,cverbsy,sent)

                                            cverbsy = replace_verbs_with_auxillary_verbs(verb,cverbsy,sent)
                                            verbx    = replaceverb_withauxillaryverb(verb,sent)
                                            if len(xcompverb) > 0: verbx = xcompverb[0]
                                            prepsxz    = replaceverb_withauxillaryverb(prepsx,sent)


                                            if len(cverbsy) > 1:
                                                for cverbsy1 in cverbsy:
                                                    verbstemp.extend([cverbsy1 + prepsxz + prepsx11.string])
                                            else:
                                                verbstemp = [verbx + prepsxz + prepsx11.string]
                                            verbsy = verbstemp        

                                            #verbstemp = [verb.string + prepsx.string + prepsx11.string]
                                            #verbsy = verbstemp

                                            if (len(subjs) > 0) and (len(objsy)) > 0  and (len(verbsy) > 0):
                                                for verb1 in verbsy:
                                                    for subj in subjs:
                                                        for obj1 in objsy:
                                                            yield (subj, verb1, obj1)   

                                        while len(objspo1) > 0 : objspo1.pop()
                                while len(prepsx1) > 0 : prepsx1.pop()    
                    while len(preps) > 0 : preps.pop()

    #verb---prep1---pcomp(verb)---dobj11/dobj12

    def subject_verb_object_pattern4(doc):

        if isinstance(doc, SpacySpan):
            sents = [doc]
        else:  # textacy.Doc or spacy.Doc
            sents = doc.sents

        for sent in sents:
            start_i = sent[0].i

            nounconjl = get_all_noun_conjunctions(sent)                   # get all noun conjunctions in the given document
            verbconjl = get_all_verb_conjunctions(sent)                   # get all verb conjunctions in the given document
            subjsa = get_subjects_of_sent(sent)                          # get all subjects in a sentence

            verbs = []
            while len(verbs) > 0 : verbs.pop()
            verbs = spacy_utils.get_main_verbs_of_sent(sent)
            """
            Algorithm logic to extract non-verbs treated as verbs
            1. Get all the nouns from the sentence
            2. for every noun check if either subject or object exists to it 
            3. If exists then add noun to the verbs list
            """


            for verb in verbs:
                """
                Algorithm logic to extract subjects of the verb

                1. Check for any subjects exists for the given verb
                2. If no subjects exists then get the main subject of the sentence as the subject
                3. check if any conjunctions exists to the subject, if conjunctions exist add conjunctions to the subject
                4. check if any prepositions exists for the subjects, if exists then extract prepositional subjects to the sentence
                       check for any conjunctions exists for prepositional objects, if present then add all the subjects to subject list
                """

                subjs = getsubjects_ofthe_verb(verb,sent)

                """
                   Algorithm logic to extract objects of the verb

                   1. Check for any objects exists for the given verb
                   2. check if any conjunctions exists to the object, if conjunctions exist add conjunctions to the object list
                   3. check if any prepositions exists for the objects, if exists then extract prepositional objects to the sentence
                      check for any conjunctions exists for prepositional objects, if present then add all the objects to object list
                """               
                xcompverb = getxcomp_ofthe_verb(verb,sent)   # get xcomp (open clausal compliment) of verb if exist
                objs = []
                objsy = []
                verbsy = []
                preps = []
                prepsx1 = []
                objspo1 = []
                verbstemp = []
                cverbsy = []

                # check if verb is having multiple prepositions which are having pcomp and is a verb, which again have prepositions and prepositional objects and its conjunctions
                preps = _get_prepositions(verb)                    # check for any prepositions exist for that object
                if len(preps) > 0:                                 # check atleast one preposition found
                    for prepsx in preps:
                            prepsx1 = [tok for tok in prepsx.rights
                                            if tok.dep_  in ['pcomp']]
                            if len(prepsx1) > 0:                                 # check atleast one preposition found
                                for prepsx11 in prepsx1:
                                    if prepsx11.pos == VERB:
                                        objspo1 = spacy_utils.get_objects_of_verb(prepsx11)    
                                        if len(objspo1) > 0:                              # check for atleast one prepositional object found
                                            for objspox1 in objspo1:
                                                while len(objsy) > 0 : objsy.pop()
                                                objsy.extend(objspo1)
                                                objs11 = getobjconjtest(objspox1,nounconjl)          # get all conjunctions to given nounlist 
                                                if len(objs11) > 0:                         # if any conjunctions found for the object
                                                    objsy.extend(objs11)                           # replace retrieved objects to objs list 
                                                    objsy = list(set())
                                                objsy = replace_objs_with_compound_objs(objsy,sent)

                                                """ Replace verbs with conjunction verbs
                                                """
                                                while len(cverbsy) > 0 : cverbsy.pop()
                                                while len(verbstemp) > 0 : verbstemp.pop()
                                                conjverb = getobjconjtest(verb,verbconjl)
                                                conjnoun = getobjconjtest(verb,nounconjl)
                                                if len(conjverb) > 0:
                                                    cverbsy.extend(conjverb)
                                                if len(conjnoun) > 0:
                                                    cverbsy.extend(conjnoun)

                                                cverbsy = verbswithdobjs(verb,cverbsy,sent)

                                                cverbsy = replace_verbs_with_auxillary_verbs(verb,cverbsy,sent)
                                                verbx    = replaceverb_withauxillaryverb(verb,sent)
                                                if len(xcompverb) > 0: verbx = xcompverb[0]
                                                prepsx11z= replaceverb_withauxillaryverb(prepsx11,sent)

                                                if len(cverbsy) > 1:
                                                    for cverbsy1 in cverbsy:
                                                        verbstemp.extend([cverbsy1 + prepsx.string + prepsx11z])
                                                else:
                                                    verbstemp = [verbx + prepsx.string + prepsx11z]
                                                verbsy = verbstemp        

                                                #verbstemp = [verb.string + prepsx.string + prepsx11.string]
                                                #verbsy = verbstemp

                                                if (len(subjs) > 0) and (len(objsy)) > 0  and (len(verbsy) > 0):
                                                    for verb1 in verbsy:
                                                        for subj in subjs:
                                                            for obj1 in objsy:
                                                                yield (subj, verb1, obj1)   

                                            while len(objspo1) > 0 : objspo1.pop()
                                while len(prepsx1) > 0 : prepsx1.pop()    
                    while len(preps) > 0 : preps.pop()        



    #verb---attr---prep1---pobj11/pobj12

    def subject_verb_object_pattern5(doc):

        if isinstance(doc, SpacySpan):
            sents = [doc]
        else:  # textacy.Doc or spacy.Doc
            sents = doc.sents

        for sent in sents:
            start_i = sent[0].i

            nounconjl = get_all_noun_conjunctions(sent)                   # get all noun conjunctions in the given document
            verbconjl = get_all_verb_conjunctions(sent)                   # get all verb conjunctions in the given document
            subjsa = get_subjects_of_sent(sent)                          # get all subjects in a sentence

            verbs = []
            while len(verbs) > 0 : verbs.pop()
            verbs = spacy_utils.get_main_verbs_of_sent(sent)
            """
            Algorithm logic to extract non-verbs treated as verbs
            1. Get all the nouns from the sentence
            2. for every noun check if either subject or object exists to it 
            3. If exists then add noun to the verbs list
            """


            for verb in verbs:
                """
                Algorithm logic to extract subjects of the verb

                1. Check for any subjects exists for the given verb
                2. If no subjects exists then get the main subject of the sentence as the subject
                3. check if any conjunctions exists to the subject, if conjunctions exist add conjunctions to the subject
                4. check if any prepositions exists for the subjects, if exists then extract prepositional subjects to the sentence
                       check for any conjunctions exists for prepositional objects, if present then add all the subjects to subject list
                """

                subjs = getsubjects_ofthe_verb(verb,sent)

                """
                   Algorithm logic to extract objects of the verb

                   1. Check for any objects exists for the given verb
                   2. check if any conjunctions exists to the object, if conjunctions exist add conjunctions to the object list
                   3. check if any prepositions exists for the objects, if exists then extract prepositional objects to the sentence
                      check for any conjunctions exists for prepositional objects, if present then add all the objects to object list
                """               
                xcompverb = getxcomp_ofthe_verb(verb,sent)   # get xcomp (open clausal compliment) of verb if exist
                objs = []
                objsy = []
                verbsy = []
                attrx = []
                preps = []
                objspo = [] 
                verbstemp = []  
                cverbsy = []
                # check if verb is having attribute followed by preposition and with preposition objects
                attrx = [tok for tok in verb.rights
                            if tok.dep_  in ['attr']]
                if len(attrx) > 0:                                 # check atleast one preposition found
                    for attrx1 in attrx:

                            preps = _get_prepositions(attrx1)       
                            if len(preps) > 0:                                 # check atleast one preposition found
                                for prepsx in preps:
                                        objspo = getprepobjects(prepsx) 
                                        if len(objspo) > 0:                              # check for atleast one prepositional object found
                                            for objspo1 in objspo:
                                                while len(objsy) > 0 : objsy.pop()
                                                objsy.extend(objspo)
                                                objs11 = getobjconjtest(objspo1,nounconjl)          # get all conjunctions to given nounlist 
                                                if len(objs11) > 0:                         # if any conjunctions found for the object
                                                    objsy.extend(objs11)                           # replace retrieved objects to objs list 
                                                    objsy = list(set(objsy))
                                                objsy = replace_objs_with_compound_objs(objsy,sent)

                                                """ Replace verbs with conjunction verbs
                                                """
                                                while len(cverbsy) > 0 : cverbsy.pop()
                                                while len(verbstemp) > 0 : verbstemp.pop()
                                                conjverb = getobjconjtest(verb,verbconjl)
                                                conjnoun = getobjconjtest(verb,nounconjl)
                                                if len(conjverb) > 0:
                                                    cverbsy.extend(conjverb)
                                                if len(conjnoun) > 0:
                                                    cverbsy.extend(conjnoun)

                                                cverbsy = verbswithdobjs(verb,cverbsy,sent)

                                                cverbsy = replace_verbs_with_auxillary_verbs(verb,cverbsy,sent)
                                                attrx1z  = replaceobj_withcompoundobj(attrx1,sent)
                                                verbx    = replaceverb_withauxillaryverb(verb,sent)
                                                if len(xcompverb) > 0: verbx = xcompverb[0]

                                                if len(cverbsy) > 1:
                                                    for cverbsy1 in cverbsy:
                                                        verbstemp.extend([cverbsy1 + attrx1z.string + prepsx.string])
                                                else:
                                                    verbstemp = [verbx + attrx1z.string + prepsx.string]
                                                verbsy = verbstemp        

                                                #verbstemp = [verb.string + attrx1.string + prepsx.string]
                                                #verbsy = verbstemp          
                                                if (len(subjs) > 0) and (len(objsy)) > 0  and (len(verbsy) > 0):
                                                    for verb1 in verbsy:
                                                        for subj in subjs:
                                                            for obj1 in objsy:
                                                                yield (subj, verb1, obj1)   

                                            while len(objspo) > 0 : objspo.pop()
                                while len(preps) > 0 : preps.pop()    
                    while len(attrx) > 0 : attrx.pop()                             



    # verb---dobj11/dobj12 or (acomp)

    def subject_verb_object_pattern6(doc):

        if isinstance(doc, SpacySpan):
            sents = [doc]
        else:  # textacy.Doc or spacy.Doc
            sents = doc.sents

        for sent in sents:
            start_i = sent[0].i

            acllist   = get_all_verb_acl(sent)
            advclverbs= get_all_verb_advcl(sent)
            nounconjl = get_all_noun_conjunctions(sent)                   # get all noun conjunctions in the given document
            verbconjl = get_all_verb_conjunctions(sent)                   # get all verb conjunctions in the given document
            subjsa = get_subjects_of_sent(sent)                          # get all subjects in a sentence

            verbs = []
            while len(verbs) > 0 : verbs.pop()
            verbs = spacy_utils.get_main_verbs_of_sent(sent)
            """
            Algorithm logic to extract non-verbs treated as verbs
            1. Get all the nouns from the sentence
            2. for every noun check if either subject or object exists to it 
            3. If exists then add noun to the verbs list
            """


            for verb in verbs:
                """
                Algorithm logic to extract subjects of the verb

                1. Check for any subjects exists for the given verb
                2. If no subjects exists then get the main subject of the sentence as the subject
                3. check if any conjunctions exists to the subject, if conjunctions exist add conjunctions to the subject
                4. check if any prepositions exists for the subjects, if exists then extract prepositional subjects to the sentence
                       check for any conjunctions exists for prepositional objects, if present then add all the subjects to subject list
                """
                subjs = getsubjects_ofthe_verb(verb,sent)

                """
                   Algorithm logic to extract objects of the verb

                   1. Check for any objects exists for the given verb
                   2. check if any conjunctions exists to the object, if conjunctions exist add conjunctions to the object list
                   3. check if any prepositions exists for the objects, if exists then extract prepositional objects to the sentence
                      check for any conjunctions exists for prepositional objects, if present then add all the objects to object list
                """               
                xcompverb = getxcomp_ofthe_verb(verb,sent)   # get xcomp (open clausal compliment) of verb if exist
                objsy = []
                preps = []
                objspo= []
                verbsy = []
                cverbsy = []

                #objs = spacy_utils.get_objects_of_verb(verb)
                objs     = get_objects_of_verb(verb) 
                objacomp = [tok for tok in verb.rights
                                    if tok.dep_  in ['acomp']]
                if len(objacomp) > 0:
                    objs.extend(objacomp)


                # if no obects exist for the verb, get objects of conjunction verbs
                if len(objs) == 0:
                    conjverb = getobjconjtest(verb,verbconjl)

                    if len(conjverb) > 1:   
                        for x in conjverb:
                            objs.extend(get_objects_of_verb(x))
                objs = replace_objs_with_compound_objs(objs,sent)

                # get  conjunction verbs 
                conjverb = getobjconjtest(verb,verbconjl)
                conjnoun = getobjconjtest(verb,nounconjl)
                if len(conjverb) > 0:
                    verbsy.extend(conjverb)
                if len(conjnoun) > 0:
                    verbsy.extend(conjnoun)
                verbsy = verbswithdobjs(verb,verbsy,sent)

                verbsyx = []
                verbsy.remove(verb)
                if len(verbsy) > 0:
                    for verbj in verbsy:
                        m1 = replaceverb_withauxillaryverb(verbj,sent)
                        verbsyx.insert(0,m1)

                if len(xcompverb) > 0: 
                    verbx = xcompverb[0]
                    verbsyx.insert(0,verbx)
                else:
                    verbx = replaceverb_withauxillaryverb(verb,sent)
                    verbsyx.insert(0,verbx)
                #verbsy = replace_verbs_with_auxillary_verbs(verbsy,sent)

                if (len(subjs) > 0) and (len(objs) > 0) and (len(verbsyx)>0):
                    for verb1 in verbsyx:
                        for subj in subjs:
                            for obj in objs:
                                yield (subj, verb1, obj)



    #verb---dobj1
    #    ---prep1---pobj11/pobj12---prep2---pobj21/pobj22


    def subject_verb_object_pattern7(doc):

        if isinstance(doc, SpacySpan):
            sents = [doc]
        else:  # textacy.Doc or spacy.Doc
            sents = doc.sents

        for sent in sents:
            start_i = sent[0].i

            nounconjl = get_all_noun_conjunctions(sent)                   # get all noun conjunctions in the given document
            verbconjl = get_all_verb_conjunctions(sent)                   # get all verb conjunctions in the given document
            subjsa = get_subjects_of_sent(sent)                          # get all subjects in a sentence

            verbs = []
            while len(verbs) > 0 : verbs.pop()
            verbs = spacy_utils.get_main_verbs_of_sent(sent)
            """
            Algorithm logic to extract non-verbs treated as verbs
            1. Get all the nouns from the sentence
            2. for every noun check if either subject or object exists to it 
            3. If exists then add noun to the verbs list
            """


            for verb in verbs:
                """
                Algorithm logic to extract subjects of the verb

                1. Check for any subjects exists for the given verb
                2. If no subjects exists then get the main subject of the sentence as the subject
                3. check if any conjunctions exists to the subject, if conjunctions exist add conjunctions to the subject
                4. check if any prepositions exists for the subjects, if exists then extract prepositional subjects to the sentence
                       check for any conjunctions exists for prepositional objects, if present then add all the subjects to subject list
                """

                subjs = getsubjects_ofthe_verb(verb,sent)

                """
                   Algorithm logic to extract objects of the verb

                   1. Check for any objects exists for the given verb
                   2. check if any conjunctions exists to the object, if conjunctions exist add conjunctions to the object list
                   3. check if any prepositions exists for the objects, if exists then extract prepositional objects to the sentence
                      check for any conjunctions exists for prepositional objects, if present then add all the objects to object list
                """               

                xcompverb = getxcomp_ofthe_verb(verb,sent)   # get xcomp (open clausal compliment) of verb if exist
                objs = get_objects_of_verb(verb) 
                objacomp = [tok for tok in verb.rights
                                    if tok.dep_  in ['acomp']]     # check if any acomp objects exist for verb
                if len(objacomp) > 0:
                    objs.extend(objacomp) 

                objsy = []
                objsyext = []
                preps = []
                objspo= []
                verbsy = []
                cverbsy = []
                objsy = []
                verbsy = []
                preps = []
                prepsx1 = []
                objspo1 = []
                verbstemp = []
                cverbsy = []


                verbstemp = []
                preps = _get_prepositions(verb)  
                if len(objs) > 0 and len(preps) > 0 :
                    for objsx in objs:
                        if len(preps) > 0:                                 # check atleast one preposition found
                            for prepsx in preps:
                                objspo = getprepobjects(prepsx)    
                                if len(objspo) > 0:                              # check for atleast one prepositional object found
                                    for objspo1 in objspo:
                                        while len(objsyext) > 0 : objsyext.pop()
                                        while len(objsy) > 0 : objsy.pop()
                                        objsy.extend(objspo)
                                        objs11 = getobjconjtest(objspo1,nounconjl)          # get all conjunctions to given nounlist 
                                        if len(objs11) > 0:                         # if any conjunctions found for the object
                                            objsy.extend(objs11)                           # replace retrieved objects to objs list 
                                            objsy = list(set(objsy))
                                        """start Check if prepositional objects is having prepositions and its objects
                                        """

                                        for objsypr in objsy:
                                            objsyprep = _get_prepositions(objsypr)
                                            if len(objsyprep) > 0: 
                                                for objsyprob in objsyprep:
                                                    objsyprobjs = getprepobjects(objsyprob)
                                                    if len(objsyprobjs) > 0:
                                                        for objsyprobjsy in objsyprobjs:
                                                            objsyext.extend(objsyprobjs)
                                                            objstt11 = getobjconjtest(objsyprobjsy,nounconjl)
                                                            if len(objstt11) > 0:
                                                                objsyext.extend(objstt11) 
                                                                objsyext = list(set(objstt11))
                                        """End Check if prepositional objects is having prepositions and its objects
                                        """                        
                                        objsy = replace_objs_with_compound_objs(objsy,sent)
                                        objsyext = replace_objs_with_compound_objs(objsyext,sent)

                                        """ Replace verbs with conjunction verbs
                                        """
                                        while len(cverbsy) > 0 : cverbsy.pop()
                                        while len(verbstemp) > 0 : verbstemp.pop()
                                        conjverb = getobjconjtest(verb,verbconjl)
                                        conjnoun = getobjconjtest(verb,nounconjl)
                                        if len(conjverb) > 0:
                                            cverbsy.extend(conjverb)
                                        if len(conjnoun) > 0:
                                            cverbsy.extend(conjnoun)

                                        cverbsy = verbswithdobjs(verb,cverbsy,sent)

                                        cverbsy  = replace_verbs_with_auxillary_verbs(verb,cverbsy,sent)
                                        objsxz   = replaceobj_withcompoundobj(objsx,sent)
                                        verbx    = replaceverb_withauxillaryverb(verb,sent)
                                        if len(xcompverb) > 0: verbx = xcompverb[0]
                                        #objsyprv = replaceobj_withcompoundobj(objsypr,sent)

                                        if len(cverbsy) > 1:
                                            for cverbsy1 in cverbsy:
                                                verbstemp.extend([cverbsy1 + objsxz.string + prepsx.string])
                                        else:
                                            verbstemp = [verbx + objsxz.string + prepsx.string]
                                        verbsy = verbstemp        

                                        #verbstemp = [verb.string + objsx.string + prepsx.string]
                                        #verbsy = verbstemp          

                                        if (len(subjs) > 0) and (len(objsy)) > 0  and (len(verbsy) > 0):
                                            for verb1 in verbsy:
                                                for subj in subjs:
                                                    for obj1 in objsy:
                                                        yield (subj, verb1, obj1)   

                                        """start Check if prepositional objects is having prepositions and its objects
                                        """                                    
                                        if len(objsyext) > 0 and len(objsy) > 1:
                                            if len(cverbsy) > 1:
                                                for cverbsy1 in cverbsy:
                                                    for objsyprv in objsy:
                                                        verbstemp.extend([cverbsy1 + objsxz.string + prepsx.string + objsyprv.string + objsyprob.string])
                                            else:
                                                for objsyprv in objsy:
                                                    verbstemp.extend([verbx + objsxz.string + prepsx.string + objsyprv.string + objsyprob.string])
                                            verbsy = verbstemp   

                                        if (len(subjs) > 0) and (len(objsyext)) > 0  and (len(verbsy) > 0):
                                            for verb1 in verbsy:
                                                for subj in subjs:
                                                    for obj1 in objsyext:
                                                        yield (subj, verb1, obj1)                                           
                                        """End Check if prepositional objects is having prepositions and its objects
                                        """                        


                                    while len(objspo) > 0 : objspo.pop()
                            while len(preps) > 0 : preps.pop()    




    #verb---dobj1
    #    ---prep1---pcomp(verb)---dobj21/dobj22

    def subject_verb_object_pattern8(doc):

        if isinstance(doc, SpacySpan):
            sents = [doc]
        else:  # textacy.Doc or spacy.Doc
            sents = doc.sents

        for sent in sents:
            start_i = sent[0].i

            nounconjl = get_all_noun_conjunctions(sent)                   # get all noun conjunctions in the given document
            verbconjl = get_all_verb_conjunctions(sent)                   # get all verb conjunctions in the given document
            subjsa = get_subjects_of_sent(sent)                          # get all subjects in a sentence

            verbs = []
            while len(verbs) > 0 : verbs.pop()
            verbs = spacy_utils.get_main_verbs_of_sent(sent)
            """
            Algorithm logic to extract non-verbs treated as verbs
            1. Get all the nouns from the sentence
            2. for every noun check if either subject or object exists to it 
            3. If exists then add noun to the verbs list
            """


            for verb in verbs:
                """
                Algorithm logic to extract subjects of the verb

                1. Check for any subjects exists for the given verb
                2. If no subjects exists then get the main subject of the sentence as the subject
                3. check if any conjunctions exists to the subject, if conjunctions exist add conjunctions to the subject
                4. check if any prepositions exists for the subjects, if exists then extract prepositional subjects to the sentence
                       check for any conjunctions exists for prepositional objects, if present then add all the subjects to subject list
                """

                subjs = getsubjects_ofthe_verb(verb,sent)

                """
                   Algorithm logic to extract objects of the verb

                   1. Check for any objects exists for the given verb
                   2. check if any conjunctions exists to the object, if conjunctions exist add conjunctions to the object list
                   3. check if any prepositions exists for the objects, if exists then extract prepositional objects to the sentence
                      check for any conjunctions exists for prepositional objects, if present then add all the objects to object list
                """               
                xcompverb = getxcomp_ofthe_verb(verb,sent)   # get xcomp (open clausal compliment) of verb if exist
                objs = get_objects_of_verb(verb)
                objspo= []
                verbsy = []
                cverbsy = []
                verbstemp = []
                objsy = []
                verbsy = []
                preps = []
                prepsx1 = []
                objspo1 = []
                verbstemp = []
                cverbsy = []
                # check if verb is having multiple prepositions which are having pcomp and is a verb, which again have prepositions and prepositional objects and its conjunctions
                preps = _get_prepositions(verb)                    # check for any prepositions exist for that object
                if (len(preps) > 0) and (len(objs) > 0):                                 # check atleast one preposition found
                    for objsx in objs:
                        for prepsx in preps:
                                prepsx1 = [tok for tok in prepsx.rights
                                                if tok.dep_  in ['pcomp']]
                                if len(prepsx1) > 0:                                 # check atleast one preposition found
                                    for prepsx11 in prepsx1:
                                        if prepsx11.pos == VERB:
                                            objspo1 = spacy_utils.get_objects_of_verb(prepsx11)    
                                            if len(objspo1) > 0:                              # check for atleast one prepositional object found
                                                for objspox1 in objspo1:
                                                    while len(objsy) > 0 : objsy.pop()
                                                    objsy.extend(objspo1)
                                                    objs11 = getobjconjtest(objspox1,nounconjl)          # get all conjunctions to given nounlist 
                                                    if len(objs11) > 0:                         # if any conjunctions found for the object
                                                        objsy.extend(objs11)                           # replace retrieved objects to objs list 
                                                        objsy = list(set(objsy))
                                                    objsy = replace_objs_with_compound_objs(objsy,sent)

                                                    """ Replace verbs with conjunction verbs
                                                    """
                                                    while len(cverbsy) > 0 : cverbsy.pop()
                                                    while len(verbstemp) > 0 : verbstemp.pop()
                                                    conjverb = getobjconjtest(verb,verbconjl)
                                                    conjnoun = getobjconjtest(verb,nounconjl)
                                                    if len(conjverb) > 0:
                                                        cverbsy.extend(conjverb)
                                                    if len(conjnoun) > 0:
                                                        cverbsy.extend(conjnoun)

                                                    cverbsy = verbswithdobjs(verb,cverbsy,sent)

                                                    cverbsy  = replace_verbs_with_auxillary_verbs(verb,cverbsy,sent)
                                                    objsxz   = replaceobj_withcompoundobj(objsx,sent)
                                                    prepsx11z= replaceverb_withauxillaryverb(prepsx11,sent)
                                                    verbx    = replaceverb_withauxillaryverb(verb,sent)
                                                    if len(xcompverb) > 0: verbx = xcompverb[0]

                                                    if len(cverbsy) > 1:
                                                        for cverbsy1 in cverbsy:
                                                            verbstemp.extend([cverbsy1 + objsxz.string + prepsx.string + prepsx11z])
                                                    else:
                                                        verbstemp = [verbx + objsxz.string + prepsx.string + prepsx11z ]

                                                    verbsy = verbstemp 

                                                    #verbstemp = [verb.string + objsx.string + prepsx.string + prepsx11.string]
                                                    #verbsy = verbstemp

                                                    if (len(subjs) > 0) and (len(objsy)) > 0  and (len(verbsy) > 0):
                                                        for verb1 in verbsy:
                                                            for subj in subjs:
                                                                for obj1 in objsy:
                                                                    yield (subj, verb1, obj1)   

                                                while len(objspo1) > 0 : objspo1.pop()
                                    while len(prepsx1) > 0 : prepsx1.pop()    
                        while len(preps) > 0 : preps.pop()        





    #verb---dobj1
    #    ---prep1---pcomp(verb)---prep2---pobj21/pobj22

    def subject_verb_object_pattern9(doc):

        if isinstance(doc, SpacySpan):
            sents = [doc]
        else:  # textacy.Doc or spacy.Doc
            sents = doc.sents

        for sent in sents:
            start_i = sent[0].i

            nounconjl = get_all_noun_conjunctions(sent)                   # get all noun conjunctions in the given document
            verbconjl = get_all_verb_conjunctions(sent)                   # get all verb conjunctions in the given document
            subjsa = get_subjects_of_sent(sent)                          # get all subjects in a sentence

            verbs = []
            while len(verbs) > 0 : verbs.pop()
            verbs = spacy_utils.get_main_verbs_of_sent(sent)
            """
            Algorithm logic to extract non-verbs treated as verbs
            1. Get all the nouns from the sentence
            2. for every noun check if either subject or object exists to it 
            3. If exists then add noun to the verbs list
            """


            for verb in verbs:
                """
                Algorithm logic to extract subjects of the verb

                1. Check for any subjects exists for the given verb
                2. If no subjects exists then get the main subject of the sentence as the subject
                3. check if any conjunctions exists to the subject, if conjunctions exist add conjunctions to the subject
                4. check if any prepositions exists for the subjects, if exists then extract prepositional subjects to the sentence
                       check for any conjunctions exists for prepositional objects, if present then add all the subjects to subject list
                """

                subjs = getsubjects_ofthe_verb(verb,sent)

                """
                   Algorithm logic to extract objects of the verb

                   1. Check for any objects exists for the given verb
                   2. check if any conjunctions exists to the object, if conjunctions exist add conjunctions to the object list
                   3. check if any prepositions exists for the objects, if exists then extract prepositional objects to the sentence
                      check for any conjunctions exists for prepositional objects, if present then add all the objects to object list
                """               
                xcompverb = getxcomp_ofthe_verb(verb,sent)   # get xcomp (open clausal compliment) of verb if exist
                objs = get_objects_of_verb(verb)
                objspo= []
                verbsy = []
                cverbsy = []
                verbstemp = []
                objsy = []
                verbsy = []
                preps = []
                prepsx1 = []
                objspo1 = []
                verbstemp = []
                cverbsy = []
                # check if verb is having multiple prepositions which are having pcomp and is a verb, which again have prepositions and prepositional objects and its conjunctions
                preps = _get_prepositions(verb)                    # check for any prepositions exist for that object
                if (len(preps) > 0) and (len(objs) > 0):                                 # check atleast one preposition found
                    for objsx in objs:
                        for prepsx in preps:
                                prepsx1 = [tok for tok in prepsx.rights
                                                if tok.dep_  in ['pcomp']]
                                if len(prepsx1) > 0:                                 # check atleast one preposition found
                                    for prepsx11 in prepsx1:
                                        if prepsx11.pos == VERB:
                                            prepspo1 = _get_prepositions(prepsx11) 
                                            #objspo1 = spacy_utils.get_objects_of_verb(prepsx11)    
                                            if len(prepspo1) > 0:                              # check for atleast one prepositional object found
                                                for prepspo11 in prepspo1:    
                                                    objspo1 = spacy_utils.get_objects_of_verb(prepspo11)
                                                    if len(objspo1) > 0:
                                                        for objspox1 in objspo1:
                                                            while len(objsy) > 0 : objsy.pop()
                                                            objsy.extend(objspo1)
                                                            objs11 = getobjconjtest(objspox1,nounconjl)          # get all conjunctions to given nounlist 
                                                            if len(objs11) > 0:                         # if any conjunctions found for the object
                                                                objsy.extend(objs11)                           # replace retrieved objects to objs list 
                                                                objsy = list(set(objsy))
                                                            objsy = replace_objs_with_compound_objs(objsy,sent)

                                                            """ Replace verbs with conjunction verbs
                                                            """
                                                            while len(cverbsy) > 0 : cverbsy.pop()
                                                            while len(verbstemp) > 0 : verbstemp.pop()
                                                            conjverb = getobjconjtest(verb,verbconjl)
                                                            conjnoun = getobjconjtest(verb,nounconjl)
                                                            if len(conjverb) > 0:
                                                                cverbsy.extend(conjverb)
                                                            if len(conjnoun) > 0:
                                                                cverbsy.extend(conjnoun)

                                                            cverbsy = verbswithdobjs(verb,cverbsy,sent)

                                                            cverbsy  = replace_verbs_with_auxillary_verbs(verb,cverbsy,sent)
                                                            objsxz   = replaceobj_withcompoundobj(objsx,sent)
                                                            prepsx11z= replaceverb_withauxillaryverb(prepsx11,sent)
                                                            verbx    = replaceverb_withauxillaryverb(verb,sent)
                                                            if len(xcompverb) > 0: verbx = xcompverb[0]

                                                            if len(cverbsy) > 1:
                                                                for cverbsy1 in cverbsy:
                                                                    verbstemp.extend([cverbsy1 + objsxz.string + prepsx.string + prepsx11z  + prepspo11.string])
                                                            else:
                                                                verbstemp = [verbx + objsxz.string + prepsx.string + prepsx11z + prepspo11.string]

                                                            verbsy = verbstemp 

                                                            #verbstemp = [verb.string + objsx.string + prepsx.string + prepsx11.string]
                                                            #verbsy = verbstemp

                                                            if (len(subjs) > 0) and (len(objsy)) > 0  and (len(verbsy) > 0):
                                                                for verb1 in verbsy:
                                                                    for subj in subjs:
                                                                        for obj1 in objsy:
                                                                            yield (subj, verb1, obj1)   

                                                    while len(objspo1) > 0 : objspo1.pop()
                                    while len(prepsx1) > 0 : prepsx1.pop()    
                        while len(preps) > 0 : preps.pop()        





    #verb---prep1---pcomp(verb)---prep2---pobj21/pobj22
    #    

    def subject_verb_object_pattern10(doc):

        if isinstance(doc, SpacySpan):
            sents = [doc]
        else:  # textacy.Doc or spacy.Doc
            sents = doc.sents

        for sent in sents:
            start_i = sent[0].i

            nounconjl = get_all_noun_conjunctions(sent)                   # get all noun conjunctions in the given document
            verbconjl = get_all_verb_conjunctions(sent)                   # get all verb conjunctions in the given document
            subjsa = get_subjects_of_sent(sent)                          # get all subjects in a sentence

            verbs = []
            while len(verbs) > 0 : verbs.pop()
            verbs = spacy_utils.get_main_verbs_of_sent(sent)
            """
            Algorithm logic to extract non-verbs treated as verbs
            1. Get all the nouns from the sentence
            2. for every noun check if either subject or object exists to it 
            3. If exists then add noun to the verbs list
            """


            for verb in verbs:
                """
                Algorithm logic to extract subjects of the verb

                1. Check for any subjects exists for the given verb
                2. If no subjects exists then get the main subject of the sentence as the subject
                3. check if any conjunctions exists to the subject, if conjunctions exist add conjunctions to the subject
                4. check if any prepositions exists for the subjects, if exists then extract prepositional subjects to the sentence
                       check for any conjunctions exists for prepositional objects, if present then add all the subjects to subject list
                """

                subjs = getsubjects_ofthe_verb(verb,sent)

                """
                   Algorithm logic to extract objects of the verb

                   1. Check for any objects exists for the given verb
                   2. check if any conjunctions exists to the object, if conjunctions exist add conjunctions to the object list
                   3. check if any prepositions exists for the objects, if exists then extract prepositional objects to the sentence
                      check for any conjunctions exists for prepositional objects, if present then add all the objects to object list
                """               
                xcompverb = getxcomp_ofthe_verb(verb,sent)   # get xcomp (open clausal compliment) of verb if exist
                objs = get_objects_of_verb(verb)
                objspo= []
                verbsy = []
                cverbsy = []
                verbstemp = []
                objsy = []
                verbsy = []
                preps = []
                prepsx1 = []
                objspo1 = []
                verbstemp = []
                cverbsy = []
                # check if verb is having multiple prepositions which are having pcomp and is a verb, which again have prepositions and prepositional objects and its conjunctions
                preps = _get_prepositions(verb)                    # check for any prepositions exist for that object
                if (len(preps) > 0) and (len(objs) == 0):                                 # check atleast one preposition found
                    #for objsx in objs:
                        for prepsx in preps:
                                prepsx1 = [tok for tok in prepsx.rights
                                                if tok.dep_  in ['pcomp']]
                                if len(prepsx1) > 0:                                 # check atleast one preposition found
                                    for prepsx11 in prepsx1:
                                        if prepsx11.pos == VERB:
                                            prepspo1 = _get_prepositions(prepsx11) 
                                            #objspo1 = spacy_utils.get_objects_of_verb(prepsx11)    
                                            if len(prepspo1) > 0:                              # check for atleast one prepositional object found
                                                for prepspo11 in prepspo1:    
                                                    objspo1 = spacy_utils.get_objects_of_verb(prepspo11)
                                                    if len(objspo1) > 0:
                                                        for objspox1 in objspo1:
                                                            while len(objsy) > 0 : objsy.pop()
                                                            objsy.extend(objspo1)
                                                            objs11 = getobjconjtest(objspox1,nounconjl)          # get all conjunctions to given nounlist 
                                                            if len(objs11) > 0:                         # if any conjunctions found for the object
                                                                objsy.extend(objs11)                           # replace retrieved objects to objs list 
                                                                objsy = list(set(objsy))
                                                            objsy = replace_objs_with_compound_objs(objsy,sent)

                                                            """ Replace verbs with conjunction verbs
                                                            """
                                                            while len(cverbsy) > 0 : cverbsy.pop()
                                                            while len(verbstemp) > 0 : verbstemp.pop()
                                                            conjverb = getobjconjtest(verb,verbconjl)
                                                            conjnoun = getobjconjtest(verb,nounconjl)
                                                            if len(conjverb) > 0:
                                                                cverbsy.extend(conjverb)
                                                            if len(conjnoun) > 0:
                                                                cverbsy.extend(conjnoun)

                                                            cverbsy = verbswithdobjs(verb,cverbsy,sent)

                                                            cverbsy  = replace_verbs_with_auxillary_verbs(verb,cverbsy,sent)
                                                            objsxz   = replaceobj_withcompoundobj(objsx,sent)
                                                            prepsx11z= replaceverb_withauxillaryverb(prepsx11,sent)
                                                            verbx    = replaceverb_withauxillaryverb(verb,sent)
                                                            if len(xcompverb) > 0: verbx = xcompverb[0]

                                                            if len(cverbsy) > 1:
                                                                for cverbsy1 in cverbsy:
                                                                    verbstemp.extend([cverbsy1 + prepsx.string + prepsx11z  + prepspo11.string])
                                                            else:
                                                                verbstemp = [verbx + prepsx.string + prepsx11z  + prepspo11.string]

                                                            verbsy = verbstemp 

                                                            #verbstemp = [verb.string + objsx.string + prepsx.string + prepsx11.string]
                                                            #verbsy = verbstemp

                                                            if (len(subjs) > 0) and (len(objsy)) > 0  and (len(verbsy) > 0):
                                                                for verb1 in verbsy:
                                                                    for subj in subjs:
                                                                        for obj1 in objsy:
                                                                            yield (subj, verb1, obj1)   

                                                    while len(objspo1) > 0 : objspo1.pop()
                                    while len(prepsx1) > 0 : prepsx1.pop()    
                        while len(preps) > 0 : preps.pop()        





    #verb---xcomp(verb)---prep2---pobj11/pobj12
    #                  ---dobj1/dobj2


    def subject_verb_object_pattern11(doc):

        if isinstance(doc, SpacySpan):
            sents = [doc]
        else:  # textacy.Doc or spacy.Doc
            sents = doc.sents

        for sent in sents:
            start_i = sent[0].i

            nounconjl = get_all_noun_conjunctions(sent)                   # get all noun conjunctions in the given document
            verbconjl = get_all_verb_conjunctions(sent)                   # get all verb conjunctions in the given document
            subjsa = get_subjects_of_sent(sent)                          # get all subjects in a sentence

            verbs = []
            while len(verbs) > 0 : verbs.pop()
            verbs = spacy_utils.get_main_verbs_of_sent(sent)
            """
            Algorithm logic to extract non-verbs treated as verbs
            1. Get all the nouns from the sentence
            2. for every noun check if either subject or object exists to it 
            3. If exists then add noun to the verbs list
            """


            for verb in verbs:

                """
                Algorithm logic to extract subjects of the verb

                1. Check for any subjects exists for the given verb
                2. If no subjects exists then get the main subject of the sentence as the subject
                3. check if any conjunctions exists to the subject, if conjunctions exist add conjunctions to the subject
                4. check if any prepositions exists for the subjects, if exists then extract prepositional subjects to the sentence
                       check for any conjunctions exists for prepositional objects, if present then add all the subjects to subject list
                """

                subjs = getsubjects_ofthe_verb(verb,sent)

                """
                   Algorithm logic to extract objects of the verb

                   1. Check for any objects exists for the given verb
                   2. check if any conjunctions exists to the object, if conjunctions exist add conjunctions to the object list
                   3. check if any prepositions exists for the objects, if exists then extract prepositional objects to the sentence
                      check for any conjunctions exists for prepositional objects, if present then add all the objects to object list
                """               
                xcompverb = getxcomp_ofthe_verb(verb,sent)   # get xcomp (open clausal compliment) of verb if exist
                objs = []
                objsy = []
                verbsy = []
                objs11 = []
                objspo1 = []
                prepsx1 = []
                preps = []
                verbstemp = []
                cverbsy = []

                # check if verb is having multiple prepositions which are verbs, which again have prepositions and prepositional objects and its conjunctions
                verbsy = []
                preps = _get_prepositions(verb)                    # check for any prepositions exist for that object
                if len(preps) > 0:                                 # check atleast one preposition found
                    for prepsx in preps:
                        if prepsx.pos == VERB:
                            prepsx1 = _get_prepositions(prepsx)                    # check for any prepositions exist for that object
                            objs = get_objects_of_verb(prepsx)
                            if len(prepsx1) > 0 and len(objs) > 0:                                 # check atleast one preposition found
                                for objspx in objs:
                                    for prepsx11 in prepsx1:
                                        objspo1 = getprepobjects(prepsx11)    
                                        if len(objspo1) > 0:                              # check for atleast one prepositional object found
                                            for objspox1 in objspo1:
                                                while len(objsy) > 0 : objsy.pop()
                                                objsy.extend(objspo1)
                                                objs11 = getobjconjtest(objspox1,nounconjl)          # get all conjunctions to given nounlist 
                                                if len(objs11) > 0:                         # if any conjunctions found for the object
                                                    objsy.extend(objs11)                           # replace retrieved objects to objs list 
                                                    objsy = list(set(objsy))
                                                objsy = replace_objs_with_compound_objs(objsy,sent)

                                                """ Replace verbs with conjunction verbs
                                                """
                                                while len(cverbsy) > 0 : cverbsy.pop()
                                                while len(verbstemp) > 0 : verbstemp.pop()
                                                conjverb = getobjconjtest(verb,verbconjl)
                                                conjnoun = getobjconjtest(verb,nounconjl)
                                                if len(conjverb) > 0:
                                                    cverbsy.extend(conjverb)
                                                if len(conjnoun) > 0:
                                                    cverbsy.extend(conjnoun)

                                                cverbsy = verbswithdobjs(verb,cverbsy,sent)

                                                cverbsy = replace_verbs_with_auxillary_verbs(verb,cverbsy,sent)
                                                verbx   = replaceverb_withauxillaryverb(verb,sent)
                                                if len(xcompverb) > 0: verbx = xcompverb[0]
                                                prepsxz = replaceverb_withauxillaryverb(prepsx,sent)
                                                objspxz = replaceobj_withcompoundobj(objspx,sent)

                                                if len(cverbsy) > 1:
                                                    for cverbsy1 in cverbsy:
                                                        verbstemp.extend([cverbsy1 + prepsxz + objspxz.string  + prepsx11.string])
                                                else:
                                                    verbstemp = [verbx + prepsxz + objspxz.string + prepsx11.string]
                                                verbsy = verbstemp        

                                                #verbstemp = [verb.string + prepsx.string + prepsx11.string]
                                                #verbsy = verbstemp

                                                if (len(subjs) > 0) and (len(objsy)) > 0  and (len(verbsy) > 0):
                                                    for verb1 in verbsy:
                                                        for subj in subjs:
                                                            for obj1 in objsy:
                                                                yield (subj, verb1, obj1)   

                                            while len(objspo1) > 0 : objspo1.pop()
                                while len(prepsx1) > 0 : prepsx1.pop()    
                    while len(preps) > 0 : preps.pop()


    #verb---xcomp(verb)---dobj1/pobj2

    def subject_verb_object_pattern12(doc):

        if isinstance(doc, SpacySpan):
            sents = [doc]
        else:  # textacy.Doc or spacy.Doc
            sents = doc.sents

        for sent in sents:
            start_i = sent[0].i

            nounconjl = get_all_noun_conjunctions(sent)                   # get all noun conjunctions in the given document
            verbconjl = get_all_verb_conjunctions(sent)                   # get all verb conjunctions in the given document
            subjsa = get_subjects_of_sent(sent)                          # get all subjects in a sentence

            verbs = []
            while len(verbs) > 0 : verbs.pop()
            verbs = spacy_utils.get_main_verbs_of_sent(sent)
            """
            Algorithm logic to extract non-verbs treated as verbs
            1. Get all the nouns from the sentence
            2. for every noun check if either subject or object exists to it 
            3. If exists then add noun to the verbs list
            """


            for verb in verbs:
                """
                Algorithm logic to extract subjects of the verb

                1. Check for any subjects exists for the given verb
                2. If no subjects exists then get the main subject of the sentence as the subject
                3. check if any conjunctions exists to the subject, if conjunctions exist add conjunctions to the subject
                4. check if any prepositions exists for the subjects, if exists then extract prepositional subjects to the sentence
                       check for any conjunctions exists for prepositional objects, if present then add all the subjects to subject list
                """

                subjs = getsubjects_ofthe_verb(verb,sent)

                """
                   Algorithm logic to extract objects of the verb

                   1. Check for any objects exists for the given verb
                   2. check if any conjunctions exists to the object, if conjunctions exist add conjunctions to the object list
                   3. check if any prepositions exists for the objects, if exists then extract prepositional objects to the sentence
                      check for any conjunctions exists for prepositional objects, if present then add all the objects to object list
                """               
                xcompverb = getxcomp_ofthe_verb(verb,sent)   # get xcomp (open clausal compliment) of verb if exist
                objs = []
                objsy = []
                verbsy = []
                objs11 = []
                objspo1 = []
                prepsx1 = []
                preps = []
                verbstemp = []
                cverbsy = []

                # check if verb is having multiple prepositions which are verbs, which again have prepositions and prepositional objects and its conjunctions
                verbsy = []
                preps = _get_prepositions(verb)                    # check for any prepositions exist for that object
                if len(preps) > 0:                                 # check atleast one preposition found
                    for prepsx in preps:
                        if prepsx.pos == VERB:
                                    objs =  get_objects_of_verb(prepsx)
                                    if len(objs) > 0:                              # check for atleast one prepositional object found
                                        for objspox1 in objs:
                                            while len(objsy) > 0 : objsy.pop()
                                            objs11 = getobjconjtest(objspox1,nounconjl)          # get all conjunctions to given nounlist 
                                            if len(objs11) > 0:                         # if any conjunctions found for the object
                                                objsy.extend(objs11)                           # replace retrieved objects to objs list 
                                                objsy = list(set(objsy))
                                            objsy = replace_objs_with_compound_objs(objsy,sent)

                                            """ Replace verbs with conjunction verbs
                                            """
                                            while len(cverbsy) > 0 : cverbsy.pop()
                                            while len(verbstemp) > 0 : verbstemp.pop()
                                            conjverb = getobjconjtest(verb,verbconjl)
                                            conjnoun = getobjconjtest(verb,nounconjl)
                                            if len(conjverb) > 0:
                                                cverbsy.extend(conjverb)
                                            if len(conjnoun) > 0:
                                                cverbsy.extend(conjnoun)

                                            cverbsy = verbswithdobjs(verb,cverbsy,sent)

                                            cverbsy = replace_verbs_with_auxillary_verbs(verb,cverbsy,sent)
                                            verbx    = replaceverb_withauxillaryverb(verb,sent)
                                            if len(xcompverb) > 0: verbx = xcompverb[0]
                                            prepsxz    = replaceverb_withauxillaryverb(prepsx,sent)

                                            if len(cverbsy) > 1:
                                                for cverbsy1 in cverbsy:
                                                    verbstemp.extend([cverbsy1 + prepsxz  ])
                                            else:
                                                verbstemp = [verbx + prepsxz  ]
                                            verbsy = verbstemp        

                                            #verbstemp = [verb.string + prepsx.string + prepsx11.string]
                                            #verbsy = verbstemp

                                            if (len(subjs) > 0) and (len(objsy)) > 0  and (len(verbsy) > 0):
                                                for verb1 in verbsy:
                                                    for subj in subjs:
                                                        for obj1 in objsy:
                                                            yield (subj, verb1, obj1)   

                    while len(preps) > 0 : preps.pop()

# import pandas
# nlp = spacy.load("en")

# df = pandas.read_excel('C:\\Users\\MEI3KOR\\Desktop\\vv.xlsx', sheetname='Sheet4')
# df.head()

    tripletrows = []
    dd1 = []
    dd = []
    #document = 'rama killed ravana'
    # for index, row in df.iterrows():
    doc1 = nlp(document)
    sent = list(doc1.sents)
    for line in sent:
        line = line.text.lower()
        doc = nlp(line)
        triples1  = list(set(subject_verb_object_pattern1(doc)))
        triples2  = list(set(subject_verb_object_pattern2(doc)))
        triples3  = list(set(subject_verb_object_pattern3(doc)))
        triples4  = list(set(subject_verb_object_pattern4(doc)))
        triples5  = list(set(subject_verb_object_pattern5(doc)))
        triples6  = list(set(subject_verb_object_pattern6(doc)))
        triples7  = list(set(subject_verb_object_pattern7(doc)))
        triples8  = list(set(subject_verb_object_pattern8(doc)))
        triples9  = list(set(subject_verb_object_pattern9(doc)))
        triples10 = list(set(subject_verb_object_pattern10(doc)))
        triples11 = list(set(subject_verb_object_pattern11(doc)))
        triples12 = list(set(subject_verb_object_pattern12(doc)))
        triples   = list(set(triples1 + triples2 + triples3 + triples4  + triples5  + triples6 +
                             triples7 + triples8 + triples9 + triples10 + triples11 + triples12))
         
        dd = [list(xx) for xx in triples]
        
        Sentence  = line
        for d in dd:
            
            Entity1   = d[0]
            Relation  = d[1]
            Entity2   = d[2]
            
            tripletrows.append([Entity1, Relation, Entity2, Sentence])
            
    triplesdf = pd.DataFrame(tripletrows, columns=['Entity1', 'Relation', 'Entity2','Sentence'])
    fdf = triplesdf.drop(['Sentence'], axis=1)
    temp=[]

    for row in fdf.iterrows():
        index, data = row
        temp.append(data.tolist())
        
    return temp
    #triplesdf.to_csv('C:\\Users\\MEI3KOR\\Desktop\\demo.csv',encoding='utf-8')

#writer.save()    

In [17]:
s = "A user should be able to conduct a search by providing either restaurant name, restaurant description, restaurant address, restaurant type or restaurant menu in the free-text search field."

In [104]:
#s = 'rama killed ravana. Sita is wife of rama.'

In [8]:
s = "I will accept your offer or decline it."

In [10]:
s = "In the backyard, the dog barked and howled at the cat."

In [12]:
s = "The dog lived in the garden, but the cat, who was smarter, lived inside the house."

In [14]:
s = "I enjoyed the apple pie that you bought for me."

In [18]:
s = "Anushka is one of the many celebrities who are part of Prime Minister Narendra Modi's Swachh Bharat Abhiyan, a plan that aims to ensure positive advocacy to ensure Indians shun habits like littering on the roads or public spaces, apart from ending open defecation in the country."

In [20]:
s = "The system displays the user account information associated with this account and requests the administrator to edit the user account, password or contact email address."

In [21]:
verbieproc(s)

[['system ', 'requests to edit ', email address],
 ['system ', 'requests ', to edit],
 ['account information ', 'associated with ', account],
 ['system ', 'requests ', administrator],
 ['system ', 'requests to edit ', user account],
 ['system ', 'displays ', account information],
 ['system ', 'requests to edit ', password]]

In [90]:
df = verbieproc('rama killed ravana. Sita is wife of rama.')

In [91]:
df = df.drop(['Sentence'], axis=1)

In [92]:
temp=[]

for row in df.iterrows():
    index, data = row
    temp.append(data.tolist())

In [93]:
temp

[['rama ', 'killed ', ravana],
 ['sita ', 'is wife of ', rama],
 ['sita ', 'is ', wife]]